In [1]:
from ultralytics import YOLO


In [2]:
import matplotlib
matplotlib.use('TkAgg') 
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import Image
import numpy as np
import cv2
import json



# YOLOv8 modelini yükleyin (yolov8n varsayılan olarak yüklenir)
model = YOLO('yolov8n.pt')  # İsterseniz farklı bir model belirtebilirsiniz

# Görüntüyü yükleyin (görüntünüzün yolunu doğru şekilde belirtin)
results = model('/Users/sedenuzumculer/aractespit/otopark.png') 
parking_spaces = [
    [1320, 1900, 1934, 2100],  # 1. park yeri
    [2343, 1895, 2916, 2065],  # 2. park yeri
    [698, 1991, 1214, 2236],  # 3. park yeri
]
def box_overlaps_space(box, space, threshold=0.5):
    """Bir sınırlayıcı kutunun bir park yeri bölgesiyle örtüşüp örtüşmediğini kontrol eder."""
    space_x1, space_y1, space_x2, space_y2 = space

    box_x1, box_y1, box_x2, box_y2 = box

    x_overlap = max(0, min(box_x2, space_x2) - max(box_x1, space_x1))
    y_overlap = max(0, min(box_y2, space_y2) - max(box_y1, space_y1))

    intersection_area = x_overlap * y_overlap
    box_area = (box_x2 - box_x1) * (box_y2 - box_y1)
    space_area = (space_x2 - space_x1) * (space_y2 - space_y1)
    union_area = box_area + space_area - intersection_area

    iou = intersection_area / union_area

    return iou > threshold

empty_spaces = []
occupied_spaces = []
car_boxes = []

for result in results:
    boxes = result.boxes
    for box in boxes:
        if model.names[int(box.cls[0])] == 'car': 
            car_boxes.append(box.xyxy[0].tolist())

for space in parking_spaces:
    is_empty = True
    for result in results:
        boxes = result.boxes
        for car_box in car_boxes:
            if box_overlaps_space(car_box, space):
                is_empty = False
                break

    if is_empty:
        empty_spaces.append(space)
    else:
        occupied_spaces.append(space)


print("Boş park yerleri:")

for space in empty_spaces:
    print(space)

print("\nDolu park yerleri ve araç koordinatları:")
for i, space in enumerate(occupied_spaces):
    if i < len(car_boxes):  # İndeks hatasını önlemek için kontrol
        print(f"Park yeri: {space}, Araç kutusu: {car_boxes[i]}")



for result in results:
    img_copy = result.orig_img.copy()
   
    
    for space in empty_spaces:
        x1, y1, x2, y2 = space
        cv2.rectangle(img_copy, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Boş park yerleri için mavi çerçeve
    for space in occupied_spaces:
        x1, y1, x2, y2 = space
        cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Dolu park yerleri için kırmızı çerçeve
     
    for box in result.boxes:
       if model.names[int(box.cls[0])] == 'car':
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Yeşil çerçeve
            cv2.putText(img_copy, 'car: {:.2f}'.format(box.conf[0]), (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)  # Metin boyutu ve rengi ayarlandı


    plt.imshow(cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB))
    plt.show()




image 1/1 /Users/sedenuzumculer/aractespit/otopark.png: 480x640 1 person, 18 cars, 1 bus, 4 trucks, 131.0ms
Speed: 20.4ms preprocess, 131.0ms inference, 11.9ms postprocess per image at shape (1, 3, 480, 640)
Boş park yerleri:
[1320, 1900, 1934, 2100]
[2343, 1895, 2916, 2065]
[698, 1991, 1214, 2236]

Dolu park yerleri ve araç koordinatları:


: 